<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/AI_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whisper + ChatGPT

## ライブラリのインストール

In [ ]:
%pip install openai

## 音声認識のための関数 (speech_to_text)

In [ ]:
# OpenAI APIキーの設定
openai_key = "" #@param {type:"string"}

In [ ]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
import openai

openai.api_key = opanai_key

RECORD = """
  const sleep = time => new Promise(resolve => setTimeout(resolve, time))
  const b2text = blob => new Promise(resolve => {
    const reader = new FileReader()
    reader.onloadend = e => resolve(e.srcElement.result)
    reader.readAsDataURL(blob)
  })
  var record = time => new Promise(async resolve => {
    stream = await navigator.mediaDevices.getUserMedia({ audio: true })
    recorder = new MediaRecorder(stream)
    chunks = []
    recorder.ondataavailable = e => chunks.push(e.data)
    recorder.start()
    await sleep(time)
    recorder.onstop = async ()=>{
      blob = new Blob(chunks)
      text = await b2text(blob)
      resolve(text)
    }
    recorder.stop()
  })
"""

def speech_to_text(model='whisper-1', language='ja', second=5):
  filename='tmp.wav'
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (second * 1000))
  b = b64decode(s.split(',')[1])

  with open(filename, 'wb+') as fw:
    fw.write(b)

  with open(filename, "rb") as fr:
    transcription = openai.Audio.transcribe(
        model=model, 
        file=fr, 
        language=language
    )
    return transcription['text']

## 動作確認

In [ ]:
print("何か喋ってください（５秒）")
text = speech_to_text()
print(text)

## ChatGPT との連携

In [ ]:
# 初期のプロンプト
messages = [
  {"role": "system", "content": "あなたはオペレータの振る舞いをするボットです。レストラン予約を完遂してください。エリア(渋谷、新宿、代々木)、料理(ラーメン、焼き肉、中華)、日付、時間を聞いて予約を確定してください。"},
  {"role": "user", "content": "こんにちは"},
]

while True:
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", 
    messages=messages, 
    stream = True
  )

  #bot_speech = completion.choices[0].message.content
  #print(f"オペレータ: {bot_speech}\n")
  response_text = ""
  for chunk in completion:
    if chunk is not None:
        content = chunk['choices'][0]['delta'].get('content')
        if content is not None:
            response_text += content
            print(content, end='', flush=True)
  print("")
  bot_speech = response_text

  while True:
    print("音声入力中（５秒） ... ")
    user_speech = speech_to_text()
    if user_speech != '':
      break
  print(f"あなた: {user_speech}")

  messages.extend([
      {"role": "assistant", "content": bot_speech},
      {"role": "user", "content": user_speech}
  ])
  
  if user_speech == 'おしまい':
    print("終了します。")
    break
